In [1]:
import numpy as np
import torch, re
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from __future__ import print_function
import numpy as np
import librosa

def tag_processing_audio(file_type):
    PATH = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/"
    FILE = "translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_46856990.transcription.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_24474003.transcription.txt"

    from pathlib import Path

    pathlist = Path(PATH+file_type).glob('**/*.wav')
    tag_list = []
    #print(pathlist)
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        #print(path_in_str)
        #print(path_in_str.split("/")[-1:][0])
        FILE2 = path_in_str.split("/")[-1:][0]
        
        # 1. Get the file path to the included audio example
        #filename = "/home/va/Kathy/scratch/src/MATERIAL_BASE-1B_10171280.wav"
        
        filename = PATH+file_type + FILE2
        # 2. Load the audio as a waveform `y`
        #    Store the sampling rate as `sr`
        print(filename)
        y, sr = librosa.load(filename, res_type='kaiser_fast')

        # 3. Run the default beat tracker
        tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T,axis=0) 

        oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=512)
        tempograms = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=512)
        #tempograms = np.mean(librosa.feature.tempogram(y=y, sr=sr, n_mfcc=40).T,axis=0) 

        #print('Estimated tempo: {:.2f} beats per minute'.format(tempo))
        np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
        #print(mfccs, tempograms)

        print(mfccs.shape, tempograms.shape)

        # 4. Convert the frame indices of beat events into timestamps
        beat_times = librosa.frames_to_time(beat_frames, sr=sr)

        librosa.output.times_csv(PATH+"processed_beats/" + FILE2 + ".txt", beat_times)
        librosa.output.times_csv(PATH+"processed_mfccs/" + FILE2 + ".txt", mfccs) #non-normalized
        #librosa.output.times_csv(PATH+"processed_tempo/" + FILE2 + ".txt", ''.join(x for x in tempograms.tolist())) #tempo
        with open(PATH+"processed_tempo/" + FILE2 + ".txt", 'w') as outfile:
            outfile.write('# Array shape: {0}\n'.format(tempograms.shape))
            # Iterating through a ndimensional array produces slices along
            # the last axis. This is equivalent to data[i,:,:] in this case
            for data_slice in tempograms:
                # The formatting string indicates that I'm writing out
                # the values in left-justified columns 7 characters in width
                # with 2 decimal places.  
                np.savetxt(outfile, data_slice, fmt='%-7.4f')
                # Writing out a break to indicate different slices...
                outfile.write('# New slice\n')
        
        '''
        with open(PATH+file_type + FILE2) as file:
            cnt = 1
            pat = r'(?<=\<).+?(?=\>)'
            #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0])
            for line in file:
                tags = re.findall(pat, line)
        
                for i in range(len(line[:-1])):
                    #print(i)
                    if(line[i] == '<'):
                        new_tag = ""
                        for j in range(i+1, len(line[:-1])):
                            if(line[j] == '>'):
                                i = j
                                if(not new_tag.isdigit() and (new_tag not in tag_list)):
                                    tag_list.append(new_tag)
                                    #print(new_tag)
                                #tag_list[new_tag] = True
                                break
                            else:
                                new_tag = new_tag + line[j]

                cnt = cnt + 1
                #break
            #break
        '''
    #print(tag_list)
    return tag_list

tag_processing_audio("src/")


/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_77097070.wav
(40,) (384, 13960)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_41171885.wav
(40,) (384, 3493)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_88100113.wav
(40,) (384, 5177)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_81447573.wav
(40,) (384, 3362)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_31089786.wav
(40,) (384, 2863)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_57086308.wav
(40,) (384, 3005)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_41581943.wav
(40,) (384, 17564)
/Use

(40,) (384, 2583)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_62980994.wav
(40,) (384, 9807)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_38244746.wav
(40,) (384, 12816)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_23555478.wav
(40,) (384, 8162)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_50599196.wav
(40,) (384, 3644)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_30471970.wav
(40,) (384, 7238)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_24474003.wav
(40,) (384, 10415)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_14493866.wav
(40,

(40,) (384, 9715)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_96279348.wav
(40,) (384, 4908)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_10760160.wav
(40,) (384, 6475)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_52960427.wav
(40,) (384, 10460)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_13843140.wav
(40,) (384, 1659)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_10675424.wav
(40,) (384, 8471)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_79504607.wav
(40,) (384, 7966)
/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/src/MATERIAL_BASE-1A_67137459.wav
(40,)

[]

In [2]:
def tag_processing(file_type):
    PATH = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/"
    FILE = "translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_46856990.transcription.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_24474003.transcription.txt"

    from pathlib import Path

    pathlist = Path(PATH+file_type).glob('**/*.txt')
    tag_list = []
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        #print(path_in_str)
        #print(path_in_str.split("/")[-1:][0])
        FILE2 = path_in_str.split("/")[-1:][0]

        with open(PATH+file_type + FILE2) as file:
            cnt = 1
            pat = r'(?<=\<).+?(?=\>)'
            #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0])
            for line in file:
                tags = re.findall(pat, line)
                '''
                if(len(tags) == 0):
                    print("Len=",  len(tags), line)
                    #print(line[:-1])
                    #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0], tags)
                    continue
                #print("New Tags")
                print(line[:-1])
                '''

                for i in range(len(line[:-1])):
                    #print(i)
                    if(line[i] == '<'):
                        new_tag = ""
                        for j in range(i+1, len(line[:-1])):
                            if(line[j] == '>'):
                                i = j
                                if(not new_tag.isdigit() and (new_tag not in tag_list)):
                                    tag_list.append(new_tag)
                                    #print(new_tag)
                                #tag_list[new_tag] = True
                                break
                            else:
                                new_tag = new_tag + line[j]

                '''
                if("<no-speech>" in line):
                    continue;
                elif("<sta>" in line):
                    print(cnt, "Start: " + line.split("<sta>",1)[1][:-1])
                elif("<int>" in line):
                    print(cnt, "Interrupt: " + line.split("<int>",1)[1][:-1])
                '''
                cnt = cnt + 1
                #break
            #break
    #print(tag_list)
    return tag_list

transcription_tag_list = tag_processing("transcription/")
translation_tag_list = tag_processing("translation/")

transcription_tag_list.sort()
translation_tag_list.sort()

print(transcription_tag_list)
print(translation_tag_list)

['breath', 'cough', 'foreign', 'hes', 'int', 'laugh', 'lipsmack', 'misc', 'no-speech', 'overlap', 'sta']
['breath', 'cough', 'foreign', 'hes', 'int', 'laugh', 'lipsmack', 'misc', 'no-speech', 'overlap', 'sta']


In [ ]:
##############Ignore Kaldi
#Kaldi Swahili dataset

In [5]:
def collated_data():
    PATH = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/"
    FILE = "translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_46856990.transcription.txt"
    FILE1 = "transcription/MATERIAL_BASE-1A_24474003.transcription.txt"

    from pathlib import Path
    
    ret = ""
    repeat = 1
    pathlist = Path(PATH+"transcription/").glob('**/*.txt')
    tag_list = []
    for path in pathlist:
        # because path is object not string
        path_in_str = str(path)
        #print(path_in_str)
        #print(path_in_str.split("/")[-1:][0])
        FILE2 = path_in_str.split("/")[-1:][0]

        #print(PATH+"transcription/" + FILE2)
        #print(PATH+"transcription/" + FILE2.replace("transcription", "translation.eng"))
        with open(PATH+"transcription/" + FILE2) as file1, open(PATH+"translation/" + FILE2.replace("transcription", "translation.eng")) as file2:
            cnt = 1
            pat = r'(?<=\<).+?(?=\>)'
            #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0])
            cur_ret = ""
            for line in file1:
                #print(line)
                for word in line[:-1].split():
                    if(word[0] == "<"):
                        for i in range(repeat):
                            ret = ret + (cur_ret + " ")
                        repeat = 1
                        cur_ret = ""
                        if(word[1:-1] == "int" or word[1:-1] == "lipsmack" or word[1:-1] == "sta" or word[1:-1] == "hes" ):
                            ret = ret + "\n\n"
                        if(word[1:-1] == "cough" or word[1:-1] == "int" or word[1:-1] == "lipsmack" or word[1:-1] == "laugh"):
                            repeat = 2
                    else:
                        cur_ret = cur_ret + " " + word
                
                ret1=""
                cnt = 1
                pat = r'(?<=\<).+?(?=\>)'
                #print(cnt, (PATH+"transcription/" + FILE2).split("/")[-1:][0])
                cur_ret = ""
                for line in file2:
                    #print(line)
                    for word in line[:-1].split():
                        if(word[0] == "<"):
                            for i in range(repeat):
                                ret1 = ret1 + (cur_ret + " ")
                            repeat = 1
                            cur_ret = ""
                            if(word[1:-1] == "int" or word[1:-1] == "lipsmack" or word[1:-1] == "sta" or word[1:-1] == "hes" ):
                                ret = ret + "\n\n"
                            if(word[1:-1] == "cough" or word[1:-1] == "int" or word[1:-1] == "lipsmack" or word[1:-1] == "laugh"):
                                repeat = 2
                        else:
                            cur_ret = cur_ret + " " + word

                #tags = re.findall(pat, line)

                '''
                for i in range(len(line[:-1])):
                    #print(i)
                    if(line[i] == '<'):
                        new_tag = ""
                        for j in range(i+1, len(line[:-1])):
                            if(line[j] == '>'):
                                i = j
                                if(not new_tag.isdigit() and (new_tag not in tag_list)):
                                    tag_list.append(new_tag)
                                    #print(new_tag)
                                #tag_list[new_tag] = True
                                break
                            else:
                                new_tag = new_tag + line[j]
                cnt = cnt + 1
                #break
                '''
            #break
    #print(tag_list)
    return ret, ret1

text_1, text_2 = collated_data()

text_file = open("Weighted_Translated.txt", "w")
text_file.write(text_1)
text_file.close()

text_file = open("Weighted_Transcribed.txt", "w")
text_file.write(text_2)
text_file.close()

#print(text_1)
#print(text_2)
#translation_vector_list = collated_data("translation/")

#transcription_tag_list.sort()
#translation_tag_list.sort()

#print(transcription_tag_list)
#print(translation_tag_list)

In [ ]:
#import tensorflow as tf
def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words."""
    with open(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

import torchwordemb
str_english = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
#vocabulary = read_data(str_english)
#print 
#print(vocabulary[:7])

str_english = "/Users/va/Courses/Kathy/data_material/NIST-data/1A/IARPA_MATERIAL_BASE-1A/ANALYSIS1/audio/translation/MATERIAL_BASE-1A_10675424.translation.eng.txt"
#vocabulary = read_data(str_english)
#print(vocabulary[:7])

vocab, vec = torchwordemb.load_word2vec_bin("/Users/va/Courses/Kathy/Word2Vec/word2vec_model.bin")
print(vec.size())
print(vec[w2v.vocab["apple"] ] )


In [ ]:
#------------------------------------------------------------------------------------------------

#Sample GAN with normal data inputs
#The word2vec vectors from vocabulary populated above is to be injected into this piece of code (as of now, the word2vec
#crashes : Binary file is 3.66GB. Need to find a way to make this work)

#------------------------------------------------------------------------------------------------

import numpy as np
vocab_size = len(vocab)
text_length = len(text_idx)
one_hot = np.zeros(([vocab_size, text_length])
one_hot[text_idx, np.arange(text_length)] = 1

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

def get_distribution_sampler(mu, sigma):
    return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))  # Gaussian

def get_generator_input_sampler():
    return lambda m, n: torch.rand(m, n)

class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.sigmoid(self.map2(x))
        return self.map3(x)

class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map2 = nn.Linear(hidden_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.elu(self.map2(x))
        return F.sigmoid(self.map3(x))

def extract(v):
    return v.data.storage().tolist()

def stats(d):
    return [np.mean(d), np.std(d)]

def decorate_with_diffs(data, exponent):
    mean = torch.mean(data.data, 1, keepdim=True)
    mean_broadcast = torch.mul(torch.ones(data.size()), mean.tolist()[0][0])
    diffs = torch.pow(data - Variable(mean_broadcast), exponent)
    return torch.cat([data, diffs], 1)

def caller():
    # Data params
    data_mean = 4
    data_stddev = 1.25

    # Model params
    g_input_size = 1     # Random noise dimension coming into generator, per output vector
    g_hidden_size = 50   # Generator complexity
    g_output_size = 1    # size of generated output vector
    d_input_size = 100   # Minibatch size - cardinality of distributions
    d_hidden_size = 50   # Discriminator complexity
    d_output_size = 1    # Single dimension for 'real' vs. 'fake'
    minibatch_size = d_input_size

    d_learning_rate = 2e-4  # 2e-4
    g_learning_rate = 2e-4
    optim_betas = (0.9, 0.999)
    num_epochs = 30000
    print_interval = 200
    d_steps = 1  # 'k' steps in the original GAN paper. Can put the discriminator on higher training freq than generator
    g_steps = 1

    # ### Uncomment only one of these
    #(name, preprocess, d_input_func) = ("Raw data", lambda data: data, lambda x: x)
    (name, preprocess, d_input_func) = ("Data and variances", lambda data: decorate_with_diffs(data, 2.0), lambda x: x * 2)

    print("Using data [%s]" % (name))

    # ##### DATA: Target data and generator input data

    d_sampler = get_distribution_sampler(data_mean, data_stddev)
    gi_sampler = get_generator_input_sampler()
    G = Generator(input_size=g_input_size, hidden_size=g_hidden_size, output_size=g_output_size)
    D = Discriminator(input_size=d_input_func(d_input_size), hidden_size=d_hidden_size, output_size=d_output_size)
    criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
    d_optimizer = optim.Adam(D.parameters(), lr=d_learning_rate, betas=optim_betas)
    g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=optim_betas)

    for epoch in range(num_epochs):
        for d_index in range(d_steps):
            # 1. Train D on real+fake
            D.zero_grad()

            #  1A: Train D on real
            d_real_data = Variable(d_sampler(d_input_size))
            d_real_decision = D(preprocess(d_real_data))
            d_real_error = criterion(d_real_decision, Variable(torch.ones(1)))  # ones = true
            d_real_error.backward() # compute/store gradients, but don't change params

            #  1B: Train D on fake
            d_gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            d_fake_data = G(d_gen_input).detach()  # detach to avoid training G on these labels
            d_fake_decision = D(preprocess(d_fake_data.t()))
            d_fake_error = criterion(d_fake_decision, Variable(torch.zeros(1)))  # zeros = fake
            d_fake_error.backward()
            d_optimizer.step()     # Only optimizes D's parameters; changes based on stored gradients from backward()

        for g_index in range(g_steps):
            # 2. Train G on D's response (but DO NOT train D on these labels)
            G.zero_grad()

            gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            g_fake_data = G(gen_input)
            dg_fake_decision = D(preprocess(g_fake_data.t()))
            g_error = criterion(dg_fake_decision, Variable(torch.ones(1)))  # we want to fool, so pretend it's all genuine

            g_error.backward()
            g_optimizer.step()  # Only optimizes G's parameters

        if epoch % print_interval == 0:
            print("%s: D: %s/%s G: %s (Real: %s, Fake: %s) " % (epoch,
                                                                extract(d_real_error)[0],
                                                                extract(d_fake_error)[0],
                                                                extract(g_error)[0],
                                                            stats(extract(d_real_data)),
                                                            stats(extract(d_fake_data))))
caller()

In [41]:
#Fallback to Word2Vec

import pandas, itertools
from sklearn.preprocessing import OneHotEncoder

df = pandas.DataFrame((x.split() for x in text_1.split("\n")))
#one_hot = pandas.get_dummies(df[])

# two example documents

# split documents to tokens
tokens_docs = [(x.split() for x in text_1.split("\n"))]

# convert list of of token-lists to one flat list of tokens
# and then create a dictionary that maps word to id of word,
# like {A: 1, B: 2} here
all_tokens = itertools.chain.from_iterable(tokens_docs)
all_tokens_cleaned = []
for x in all_tokens:
    if(len(x)>0):
        #print(x)
        print(x)
        for i in x:
            print(i)
            all_tokens_cleaned.append(i)
        #all_tokens_cleaned.append(i[0] for i in x.split())
#print(all_tokens_cleaned)
word_to_id = {token: idx for idx, token in enumerate(set(all_tokens_cleaned))}

# convert token lists to token-id lists, e.g. [[1, 2], [2, 2]] here
token_ids = [[word_to_id[token] for token in tokens_doc] for tokens_doc in tokens_docs]

# convert list of token-id lists to one-hot representation
vec = OneHotEncoder(n_values=len(word_to_id))
X = vec.fit_transform(token_ids)

#print(X.toarray())

['(108.641-112.576)', 'kutoka', 'Dar', 'es', 'Salaam', 'Beatrice', 'Eric', 'Channel', 'moja', 'sufuri', '.', 'from', 'Dar', 'es', 'Salaam', 'Beatrice', 'Eric', 'Channel', 'Ten', '.']
(108.641-112.576)
kutoka
Dar
es
Salaam
Beatrice
Eric
Channel
moja
sufuri
.
from
Dar
es
Salaam
Beatrice
Eric
Channel
Ten
.
['Wizara', 'kama', 'Wizara', 'ina', 'dhamira', 'kweli']
Wizara
kama
Wizara
ina
dhamira
kweli
['C_C_B_R_T', 'nayo', 'ina', 'dhamira', 'ya', 'kweli', 'kuhakikisha', 'kwamba', 'tatizo']
C_C_B_R_T
nayo
ina
dhamira
ya
kweli
kuhakikisha
kwamba
tatizo
['Fistula', 'haiui', 'kama', 'magonjwa', 'mengine', 'ya', 'milipuko', 'lakini', 'ni', 'vizuri', 'tukajua', 'kwamba', 'kuna', 'kifo', 'cha', 'mwili', '0.000', 'outLine']
Fistula
haiui
kama
magonjwa
mengine
ya
milipuko
lakini
ni
vizuri
tukajua
kwamba
kuna
kifo
cha
mwili
0.000
outLine
['0.000', 'inLine', '4.470', 'inLine', 'ni', 'ile', 'ile', 'tu', 'nilikuwa', 'nakupigia', 'nayo', '4.470', 'outLine', '7.440', 'inLine', '8.700', 'inLine', '9.050', 'i

inLine
165.659
outLine
166.025
inLine
ako
tu
lakini
nilishangaa
kuona
yule
tena
--
yule
wa
['Kisii', 'tena', 'anaanza', 'kunipigiapigia', 'masimu']
Kisii
tena
anaanza
kunipigiapigia
masimu
['166.995', 'outLine', '175.547', 'outLine', '177.035', 'outLine', '178.415', 'outLine', '179.046', 'inLine', '181.435', 'inLine', '182.556', 'outLine', '184.675', 'outLine', '186.673', 'inLine', '188.057', 'outLine', '188.269', 'inLine', 'anapiga', 'tu', 'ati', 'kunisalimia', '189.165', 'outLine', '191.965', 'outLine', '192.545', 'outLine', '193.632', 'inLine', '195.125', 'inLine', '199.365', 'outLine', '199.585', 'inLine']
166.995
outLine
175.547
outLine
177.035
outLine
178.415
outLine
179.046
inLine
181.435
inLine
182.556
outLine
184.675
outLine
186.673
inLine
188.057
outLine
188.269
inLine
anapiga
tu
ati
kunisalimia
189.165
outLine
191.965
outLine
192.545
outLine
193.632
inLine
195.125
inLine
199.365
outLine
199.585
inLine
['201.425', 'outLine', '201.795', 'outLine', '204.125', 'outLine']
201.425

za
viwanja
na
moja
ya
maswali
alikuwa
ameulizwa
kuna
viwanja
vitano
ambavyo
vimepitishwa
na
shirikisho
la
soccer
ulimwenguni
FIFA
['kwamba']
kwamba
['na', 'hii', 'ndiyo', 'tatizo', 'kubwa', 'kabisa', 'sasa', 'sijui', 'vitakuwa', 'vina', 'uwezo', 'kiasi', 'gani', 'kwa', 'sababu']
na
hii
ndiyo
tatizo
kubwa
kabisa
sasa
sijui
vitakuwa
vina
uwezo
kiasi
gani
kwa
sababu
['kwamba', 'halisi', 'nzuri', 'ni']
kwamba
halisi
nzuri
ni
['hata', 'hizo', 'kubwa', 'ukiondoa', 'timu', 'ya', 'Azam', 'F_C', 'wao', 'ndiyo', 'wana', 'kiwanja', 'chao', 'kule']
hata
hizo
kubwa
ukiondoa
timu
ya
Azam
F_C
wao
ndiyo
wana
kiwanja
chao
kule
['Azam', 'complex', 'ambao', 'kimekidhi', 'vi-', '--']
Azam
complex
ambao
kimekidhi
vi-
--
['ukiondoa', 'uwanja', 'wa', 'C_C_M', 'Kirumba', 'lakini', 'katika', 'hii', 'michuano', 'ya', 'timu', 'za', 'vijana', 'ambayo', 'itafanyika', 'hapa', 'Mwanza']
ukiondoa
uwanja
wa
C_C_M
Kirumba
lakini
katika
hii
michuano
ya
timu
za
vijana
ambayo
itafanyika
hapa
Mwanza
['kwamba', 'kutakuwa', 

wa
huku
ndiyo
wanafanya
vizuri
ku
expose
kumbe
matukio
yake
ndiyo
haya
sijawahi
--
sijawahi
kuona
katika
ile
C_N_N
lita-
--
licha
ya
wanasema
ndiyo
the
first
to
know
.
tena
baada
['tena', 'hii', 'katika', 'desert', 'operation', 'storm', 'wale', 'operation', 'tena', 'hii', 'katika', 'desert', 'operation', 'storm', 'wale', 'operation', 'na', 'wao', 'ndiyo', 'first', 'to', 'know', 'na', 'ni', 'huwa', 'wanasema', 'lakini', 'unasikia', 'bwana', 'Maalim', 'huwa', 'wanasema', 'lakini']
tena
hii
katika
desert
operation
storm
wale
operation
tena
hii
katika
desert
operation
storm
wale
operation
na
wao
ndiyo
first
to
know
na
ni
huwa
wanasema
lakini
unasikia
bwana
Maalim
huwa
wanasema
lakini
['hawaweki', 'zile', 'detail']
hawaweki
zile
detail
['lakini', 'vinaheshimu', 'ethics', 'kwamba', 'habari', 'yamkini', 'ikawa', 'inatoka', 'kuna', 'watu', 'hawakubaliani', 'nao', 'lakini', 'wao', 'wanakwambia', 'lakini', 'utakapofanya', 'mbaya', 'mimi', 'nitasema', 'lakini', 'tunapokuja', 'katika', 'maswala', 

but
since
he
drilled
it
.
['mzee', 'kule', 'nyuma', 'unaweza', 'kunieleza', 'kutokana', 'na', 'ulivyokuwa', 'unasikia', 'zama', 'ndani', 'kitu', 'gani', 'ambacho', 'uli-']
mzee
kule
nyuma
unaweza
kunieleza
kutokana
na
ulivyokuwa
unasikia
zama
ndani
kitu
gani
ambacho
uli-
['lakini', 'hii', 'imesababisha']
lakini
hii
imesababisha
['kwa', 'kuwa', 'imesababisha', 'kero', 'imesababisha', 'madhara', 'fulani', 'kwa', 'Watanzania']
kwa
kuwa
imesababisha
kero
imesababisha
madhara
fulani
kwa
Watanzania
['okay', 'katika', 'mfumo', 'huo', 'wa', 'analojia', 'ilikuwepo', 'ni', 'lazima', 'mitambo', 'iwekwe', '--', 'mitambo', 'tofauti', 'iwekwe', 'sehemu', 'moja', 'ili', 'kuwawezesha', 'watu', 'wanaokaa', 'maeneo', 'hayo']
okay
katika
mfumo
huo
wa
analojia
ilikuwepo
ni
lazima
mitambo
iwekwe
--
mitambo
tofauti
iwekwe
sehemu
moja
ili
kuwawezesha
watu
wanaokaa
maeneo
hayo
['lakini', 'katika', 'huu', 'mfumo', 'wa', 'dijitali']
lakini
katika
huu
mfumo
wa
dijitali
['ambayo', 'hii', 'Satellite', 'itakuwa', '

...

majini
SUMATRA
kuidhinisha
usafiri
wa
bodaboda
pamoja
na
bajaj
kubeba
abiria
vyooni
mitaani
kutoka
jijini
Dar
es
Salaam
hii
leo
inaangazia
usafiri
wa
bodaboda
ambao
['kwa', 'wengine', 'imekuwa', 'ukombozi', 'kwa', 'wengine', 'imekuwa', 'ukombozi']
kwa
wengine
imekuwa
ukombozi
kwa
wengine
imekuwa
ukombozi
['mkubwa', 'wa', 'usafiri', 'mkubwa', 'wa', 'usafiri']
mkubwa
wa
usafiri
mkubwa
wa
usafiri
['(())', 'lazima', 'za', 'la', '(())', 'lazima', 'za', 'la']
(())
lazima
za
la
(())
lazima
za
la
['usafiri', 'huu', 'kwa', 'kweli']
usafiri
huu
kwa
kweli
['sisi', 'tunashangaa', 'serikali', 'au', 'tuseme', 'jiji', 'au']
sisi
tunashangaa
serikali
au
tuseme
jiji
au
['kwa', 'sababu', 'usafiri', 'ukiangalia', 'miundo', 'mbinu', 'yetu', 'ya', 'mji', 'huu', 'kwa', 'kweli', 'haikidhi']
kwa
sababu
usafiri
ukiangalia
miundo
mbinu
yetu
ya
mji
huu
kwa
kweli
haikidhi
['haja', 'za', 'kusema', 'kwamba', 'labda', 'mtu', 'ana', 'uwepesi', 'wa', 'kutoka']
haja
za
kusema
kwamba
labda
mtu
ana
uwepesi
wa
kutoka
['bo

ni
moja
ya
wilaya
iliyoathirika
['bila', 'kujali']
bila
kujali
['0.000', 'outLine']
0.000
outLine
['ni', 'P_P_O', 'mkubwa', '.', 'he', 'is', 'chief', 'P_P_O', '.', '0.000', 'inLine', '0.060', 'outLine']
ni
P_P_O
mkubwa
.
he
is
chief
P_P_O
.
0.000
inLine
0.060
outLine
['3.600', 'inLine']
3.600
inLine
['4.360', 'inLine', '4.360', 'inLine', '6.170', 'inLine', '10.390', 'outLine', '13.540', 'inLine', '14.666', 'outLine', '16.560', 'outLine', '16.630', 'inLine', '22.090', 'inLine', '27.410', 'outLine', '29.400', 'outLine', '34.090', 'inLine', '37.190', 'inLine', '42.800', 'outLine', '47.540', 'inLine', '51.020', 'outLine', '53.790', 'inLine', '54.500', 'outLine']
4.360
inLine
4.360
inLine
6.170
inLine
10.390
outLine
13.540
inLine
14.666
outLine
16.560
outLine
16.630
inLine
22.090
inLine
27.410
outLine
29.400
outLine
34.090
inLine
37.190
inLine
42.800
outLine
47.540
inLine
51.020
outLine
53.790
inLine
54.500
outLine
['55.550', 'outLine', '56.380', 'inLine']
55.550
outLine
56.380
inLine
['hak

295.195
inLine
295.795
outLine
296.045
inLine
297.975
outLine
299.185
inLine
300.675
outLine
halo
302.325
outLine
303.115
inLine
['304.245', 'outLine', '304.455', 'inLine', '305.313', 'outLine', '306.515', 'inLine', '307.795', 'inLine', '307.855', 'outLine', '308.225', 'inLine', '309.075', 'outLine', 'halo', '310.425', 'inLine']
304.245
outLine
304.455
inLine
305.313
outLine
306.515
inLine
307.795
inLine
307.855
outLine
308.225
inLine
309.075
outLine
halo
310.425
inLine
['310.465', 'outLine', '311.975', 'inLine', '314.035', 'outLine', '314.805', 'inLine', '315.675', 'inLine', '316.055', 'outLine', '316.105', 'inLine', '317.955', 'inLine']
310.465
outLine
311.975
inLine
314.035
outLine
314.805
inLine
315.675
inLine
316.055
outLine
316.105
inLine
317.955
inLine
['318.445', 'inLine', '318.445', 'inLine', '320.975', 'outLine', '321.328', 'inLine', '324.443', 'outLine', '326.015', 'outLine', '326.435', 'outLine', '327.053', 'inLine', '327.565', 'outLine', '329.075', 'inLine', '330.675', 'ou

mzuri
sana
juu
tulikuwa
nadhania
22.085
inLine
23.990
inLine
25.580
inLine
27.820
inLine
29.480
inLine
30.819
inLine
31.920
inLine
33.550
inLine
34.780
inLine
35.670
outLine
35.980
inLine
okay
37.620
inLine
38.710
outLine
38.960
inLine
41.770
outLine
44.320
inLine
47.550
outLine
50.330
inLine
51.240
inLine
51.990
outLine
53.590
outLine
57.180
inLine
60.750
outLine
70.500
outLine
73.890
inLine
75.790
inLine
77.580
outLine
80.135
outLine
81.890
outLine
82.705
inLine
85.220
outLine
ehhe
na
vile
tunaomba
sasa
tukichagua
tu-
--
['87.370', 'inLine', '88.450', 'inLine', '89.630', 'inLine', '91.163', 'outLine', '92.190', 'inLine', '92.430', 'outLine', '94.260', 'inLine', '96.010', 'outLine', '96.399', 'inLine', '98.420', 'inLine', '100.450', 'inLine', '102.140', 'outLine', '102.300', 'inLine', '104.770', 'inLine', '106.290', 'outLine', '111.410', 'outLine', '115.460', 'outLine', '115.976', 'inLine', '119.000', 'inLine', '119.810', 'outLine', '121.723', 'outLine', '123.080', 'outLine', '125.539

ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.